In [67]:
import numpy as np
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
from icecream import ic
from dataclasses import dataclass

## n-puzzle

In [68]:
PUZZLE_DIM = 3
action = namedtuple('Action', ['pos1', 'pos2'])

In [69]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions

def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

def goal_test(state: np.ndarray) -> bool:
    c = 1
    for i in state:
        for j in i:
            if j != c:
                return False
            c += 1
            if c == len(state)**2:
                c = 0
    return True

def stateInList(state: np.ndarray, frontier: list[np.ndarray]) -> int:
    for i, element in enumerate(frontier):
        c = 0
        for x in range(PUZZLE_DIM):
            for y in range(PUZZLE_DIM):
                if state[x][y] == element[x][y]:
                    c += 1
        if c == PUZZLE_DIM**2:
            return i
    return -1


In [70]:
RANDOMIZE_STEPS = 100_000
initial_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    #scelgo randomicamente un azione da quelle possibili e la effettuo
    initial_state = do_action(initial_state, choice(available_actions(initial_state)))
initial_state

Randomizing:   0%|          | 0/100000 [00:00<?, ?it/s]

array([[2, 8, 3],
       [6, 0, 7],
       [5, 1, 4]])

## depth-first approach

Memory consumption is high, we can't always find a solution if the number of iterations is too low

In [71]:
frontier = list()
visited_state = list()
cost_list = list()
cost = 0
state = initial_state
for step in tqdm(range(1000)):
    visited_state.append(state)
    if goal_test(state):
        break
    for a in available_actions(state):
        new_state = do_action(state, a)
        new_cost = cost + 1
        if stateInList(new_state, visited_state) == -1 and stateInList(new_state, frontier) == -1:
            frontier.append(new_state)
            cost_list.append(new_cost)
    state = frontier.pop()
    cost = cost_list.pop()
efficiency = cost / (len(frontier) + len(visited_state))
ic(goal_test(state), state, efficiency)
None

  0%|          | 0/1000 [00:00<?, ?it/s]

ic| goal_test(state): False
    state: array([[2, 8, 6],
                  [7, 1, 4],
                  [3, 5, 0]])
    efficiency: 0.5457608085345311


## breadth-first approach

Memory consumption is high, we can't always find a solution if the number of iterations is too low

In [72]:
frontier = list()
visited_state = list()
cost_list = list()
cost = 0
state = initial_state
for step in tqdm(range(1000)):
    visited_state.append(state)
    if goal_test(state):
        break
    for a in available_actions(state):
        new_state = do_action(state, a)
        new_cost = cost + 1
        if stateInList(new_state, visited_state) == -1 and stateInList(new_state, frontier) == -1:
            frontier.append(new_state)
            cost_list.append(new_cost)
    state = frontier.pop(0)
    cost = cost_list.pop(0)
efficiency = cost / (len(frontier) + len(visited_state))
ic(goal_test(state), state, efficiency)
None

  0%|          | 0/1000 [00:00<?, ?it/s]

ic| goal_test(state): False
    state: array([[3, 8, 7],
                  [2, 5, 4],
                  [6, 0, 1]])
    efficiency: 0.0069356872635561164


## A*

In [73]:
@dataclass
class Node:
    state: np.ndarray
    cost: int = 0
    quality: int = 0

def stateInNodeList(state: np.ndarray, frontier: list[Node]) -> int:
    for i, element in enumerate(frontier):
        c = 0
        for x in range(PUZZLE_DIM):
            for y in range(PUZZLE_DIM):
                if state[x][y] == element.state[x][y]:
                    c += 1
        if c == PUZZLE_DIM**2:
            return i
    return -1

def HammingDistance(state: np.ndarray) -> int:
    counter = 0
    c = 1
    for x in range(0, PUZZLE_DIM):
        for y in range(0, PUZZLE_DIM):
            if state[x][y] != c:
                counter += 1
            c += 1
            if c == PUZZLE_DIM**2:
                break
    return counter

def ManhattanDistance(state: np.ndarray) -> int:
    counter = 0
    c = 1
    for x in range(0, PUZZLE_DIM):
        for y in range(0, PUZZLE_DIM):
            if state[x][y] != c and state[x][y] != 0:
                counter += abs(x - ((state[x][y] - 1) // PUZZLE_DIM)) + abs(y - ((state[x][y] - 1) % PUZZLE_DIM))
            c += 1
    return counter

def LinearConflict(state: np.ndarray) -> int:
        conflict = 0
        size = len(state)
        # Row conflicts
        for row in range(size):
            max_val = -1
            for col in range(size):
                value = state[row][col]
                if value != 0 and (value - 1) // size == row:
                    if value > max_val:
                        max_val = value
                    else:
                        conflict += 2
        # Column conflicts
        for col in range(size):
            max_val = -1
            for row in range(size):
                value = state[row][col]
                if value != 0 and (value - 1) % size == col:
                    if value > max_val:
                        max_val = value
                    else:
                        conflict += 2
        return conflict

In [74]:
frontier = list()
visited_state = list()
state = Node(initial_state, 1 + ManhattanDistance(initial_state) + LinearConflict(initial_state), 0)
for step in tqdm(range(10000)):
    visited_state.append(state)
    if state.cost == state.quality:
        break
    for a in available_actions(state.state):
        new_state = do_action(state.state, a)
        if stateInNodeList(new_state, visited_state) == -1 and stateInNodeList(new_state, frontier) == -1:
            frontier.append(Node(new_state, state.quality + 1 + ManhattanDistance(new_state) + LinearConflict(new_state), state.quality + 1))
    frontier.sort(key=lambda i: i.cost, reverse = False)
    state = frontier.pop(0)
efficiency = state.cost / (len(frontier) + len(visited_state))
ic(initial_state, ManhattanDistance(initial_state) + LinearConflict(initial_state), goal_test(state.state), state, efficiency)
None

  0%|          | 0/10000 [00:00<?, ?it/s]

ic| initial_state: array([[2, 8, 3],
                          [6, 0, 7],
                          [5, 1, 4]])
    ManhattanDistance(initial_state) + LinearConflict(initial_state): np.int64(16)
    goal_test(state.state): True
    state: Node(state=array([[1, 2, 3],
                  [4, 5, 6],
                  [7, 8, 0]]),
                cost=20,
                quality=20)
    efficiency: 0.06042296072507553


## IDA*

In [75]:
def stateInList(state: np.ndarray, frontier: list) -> int:
    for i, element in enumerate(frontier):
        c = 0
        for x in range(PUZZLE_DIM):
            for y in range(PUZZLE_DIM):
                if state[x][y] == element[x][y]:
                    c += 1
        if c == PUZZLE_DIM**2:
            return i
    return -1

def search(path, g, bound, dirs):
    state = path[-1]
    f = g + ManhattanDistance(state) + LinearConflict(state)
    if f > bound:
        return f
    if goal_test(state):
        return True
    min = np.inf
    for a in available_actions(state):
        new_state = do_action(state, a)
        if stateInList(new_state, path) == -1:
            path.append(new_state)
            dirs.append(a)
            tmp = search(path, g+1, bound, dirs)
            if tmp == True:
                return True
            if tmp < min:
                min = tmp
            path.pop()
            dirs.pop()
    return min


In [76]:
h = ManhattanDistance(initial_state) + LinearConflict(initial_state)
bound = h
path = [initial_state]
dirs = list()
ic(initial_state, h)
while True:
    rem = search(path, 0, bound, dirs)
    if rem == True:
        ic(True)
        break
    elif rem > 100:   
        ic(False)
        break
    bound = rem
state = initial_state
for a in dirs:
    state = do_action(state, a)
quality =  len(dirs)
ic(state, quality)
None

ic| initial_state: array([[2, 8, 3],
                          [6, 0, 7],
                          [5, 1, 4]])
    h: np.int64(16)
ic| True
ic| state: array([[1, 2, 3],
                  [4, 5, 6],
                  [7, 8, 0]])
    quality: 20


### sources
- prof example
- past exercises
- slides
- forum on internet